# IR Lab SoSe 2024: Baseline Retrieval System

This jupyter notebook serves as baseline retrieval system that you can try to improve upon.
We will use the a corpus of scientific papers (title + abstracts) from the fields of information retrieval and natural language processing (the [IR Anthology](https://ir.webis.de/anthology/) and the [ACL Anthology](https://aclanthology.org/)). This serves Jupyter notebook only serves as retrieval system, i.e., it gets a set of information needs (topics) and a corpus as input and produces a run file as output. Please do evaluations in a new dedicated notebook.

### Step 1: Import Libraries

We will use [tira](https://www.tira.io/), an information retrieval shared task platform, for loading the (pre-built) retrieval index and [ir_dataset](https://ir-datasets.com/) to subsequently build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine.

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [1]:
# You only need to execute this cell if you are using Google Golab.
# If you use GitHub Codespaces, everything is already installed.
!pip3 install tira ir-datasets python-terrier

# install spacy
%pip install -U pip setuptools wheel
%pip install -U spacy
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import spacy

In [3]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


### Step 2: Load the Dataset and the Index

The type of the index object that we load is `<class 'jnius.reflect.org.terrier.structures.Index'>`, in fact a [Java class](http://terrier.org/docs/v3.6/javadoc/org/terrier/structures/Index.html) wrapped into Python. However, you do not need to worry about this: at this point, we will simply use the provided Index object to run procedures defined in Python.

In [4]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

--------
Zähle alle Entity typen


In [5]:
# entity Recognition
# Process all documents with spacy.
nlp = spacy.load("en_core_web_sm")

# count entitys
# Erstelle ein Dictionary {}
EntityTypeDict = dict()
#store all processed documents for easy random access
processed_documents_store = {}
# gehe über alle abstracts mit spacy und ließ deren Entitäten
for i in pt_dataset.get_corpus_iter():
    doc = nlp(i.get("text"))
    processed_documents_store[i.get('docno')] = nlp(i.get("text")).ents
    for ent in doc.ents:
        #print("entity:")
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)
        # wenn der Entity Type bereits im Dictionary ist dann ...
        if ent.label_ in EntityTypeDict:
            EntityTypeDict[ent.label_] += 1
        # falls nicht füge den Entity Typen in den Dictionary mit Initial 1 vorkommen    
        else: 
            EntityTypeDict[ent.label_] = 1   
print(EntityTypeDict)

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:   6%|▌         | 7141/126958 [03:49<1:00:26, 33.04it/s]

: 

In [6]:
def retain_only_entity_type_text(corpus_iter, positive_entity_types):
    ret = []
    for i in corpus_iter:
        processe_document = nlp(i.get("text"))
        text = ""
        for ent in processe_document.ents:
            if str(ent.label_) in positive_entity_types:
                text += " " + ent.text
        if len(text) > 0:
            ret += [{"docno": i["docno"], "text": text}]    
    return ret        

-----
alle Entitäten

In [7]:
indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_full_text = pt.IndexFactory.of(indexer.index(pt_dataset.get_corpus_iter()))
bm25 = pt.BatchRetrieve(index_full_text, wmodel="BM25")

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  71%|███████   | 89941/126958 [00:24<00:05, 6412.23it/s]

12:24:39.503 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:29<00:00, 4235.25it/s] 


12:24:47.570 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


----
nur ORG


In [9]:
indexer = pt.IterDictIndexer("/tmp/index-ORG", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_org_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['ORG']))))
bm25_org = pt.BatchRetrieve(index_org_text, wmodel="BM25")

KeyboardInterrupt: 

----
nur CARDINAL

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-CARDINAL", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_card_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['CARDINAL']))))
bm25_card = pt.BatchRetrieve(index_card_text, wmodel="BM25")

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [30:49<00:00, 68.64it/s] 


09:34:46.164 [ForkJoinPool-6-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (O02-2002) - further warnings are suppressed
09:34:48.117 [ForkJoinPool-6-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 21476 empty documents


----
nur PERSON

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-PERSON", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_pers_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['PERSON']))))
bm25_pers = pt.BatchRetrieve(index_pers_text, wmodel="BM25")

NameError: name 'retain_only_entity_type_text' is not defined

----
nur LANGUAGE

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-LANGUAGE", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_lang_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['LANGUAGE']))))
bm25_lang = pt.BatchRetrieve(index_lang_text, wmodel="BM25")

----
nur WORK_OF_ART

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-WORK_OF_ART", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_woa_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['WORK_OF_ART']))))
bm25_woa = pt.BatchRetrieve(index_woa_text, wmodel="BM25")

----
nur NORP

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-NORP", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_norp_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['NORP']))))
bm25_norp = pt.BatchRetrieve(index_norp_text, wmodel="BM25")

----
nur GPE

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-GPE", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_gpe_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['GPE']))))
bm25_gpe = pt.BatchRetrieve(index_gpe_text, wmodel="BM25")

----
nur PERCENT

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-PERCENT", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_perc_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['PERCENT']))))
bm25_perc = pt.BatchRetrieve(index_perc_text, wmodel="BM25")

----
nur ORDINAL

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-ORDINAL", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_ord_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['ORDINAL']))))
bm25_ord = pt.BatchRetrieve(index_ord_text, wmodel="BM25")

----
nur DATE

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-DATE", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_date_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['DATE']))))
bm25_date = pt.BatchRetrieve(index_date_text, wmodel="BM25")

----
nur QUANTITY

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-QUANTITY", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_quant_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['QUANTITY']))))
bm25_quant = pt.BatchRetrieve(index_quant_text, wmodel="BM25")

----
nur PRODUCT

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-PRODUCT", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_prod_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['PRODUCT']))))
bm25_prod = pt.BatchRetrieve(index_prod_text, wmodel="BM25")

----
nur FAC

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-FAC", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_fac_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['FAC']))))
bm25_fac = pt.BatchRetrieve(index_fac_text, wmodel="BM25")

----
nur LOC

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-LOC", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_loc_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['LOC']))))
bm25_loc = pt.BatchRetrieve(index_loc_text, wmodel="BM25")

----
nur MONEY

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-MONEY", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_money_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['MONEY']))))
bm25_money = pt.BatchRetrieve(index_money_text, wmodel="BM25")

----
nur EVENT

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-EVENT", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_event_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['EVENT']))))
bm25_event = pt.BatchRetrieve(index_event_text, wmodel="BM25")

----
nur TIME

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-TIME", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_time_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['TIME']))))
bm25_time = pt.BatchRetrieve(index_time_text, wmodel="BM25")

----
nur LAW

In [ ]:
indexer = pt.IterDictIndexer("/tmp/index-LAW", overwrite=True, stemmer='PorterStemmer', meta={'docno': 75, 'text': 4096})

index_law_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(pt_dataset.get_corpus_iter(), set(['LAW']))))
bm25_law = pt.BatchRetrieve(index_law_text, wmodel="BM25")

----
Ergbenisse

In [34]:
pt.Experiment([bm25, bm25_org, bm25_card, bm25_pers, bm25_lang, bm25_woa, bm25_norp, bm25_gpe, bm25_perc, bm25_ord, bm25_date, bm25_quant, bm25_prod, bm25_fac, bm25_loc, bm25_money, bm25_event, bm25_time, bm25_law], pt_dataset.get_topics("query"), pt_dataset.get_qrels(), eval_metrics=['ndcg_cut_10', 'P_10'])

,name,ndcg_cut_10,P_10
0,BR(BM25),0.374041,0.332353
1,BR(BM25),0.003237,0.001471


### Step 4: Create the Run


In [15]:
print('First, we have a short look at the first three topics:')

pt_dataset.get_topics('text').head(3)

First, we have a short look at the first three topics:


,qid,query
0,1,retrieval system improving effectiveness
1,2,machine learning language identification
2,3,social media detect self harm


In [20]:
print('Now we do the retrieval...')
run = bm25(pt_dataset.get_topics('text'))

print('Done. Here are the first 10 entries of the run')
run.head(10)

Now we do the retrieval...
Done. Here are the first 10 entries of the run


,qid,docid,docno,rank,score,query
0,1,94858,2004.cikm_conference-2004.47,0,15.681777,retrieval system improving effectiveness
1,1,125137,1989.ipm_journal-ir0volumeA25A4.2,1,15.047380,retrieval system improving effectiveness
2,1,125817,2005.ipm_journal-ir0volumeA41A5.11,2,14.144223,retrieval system improving effectiveness
3,1,5868,W05-0704,3,14.025748,retrieval system improving effectiveness
4,1,84876,2016.ntcir_conference-2016.90,4,13.947994,retrieval system improving effectiveness
5,1,82472,1998.sigirconf_conference-98.15,5,13.901647,retrieval system improving effectiveness
6,1,94415,2008.cikm_conference-2008.183,6,13.808208,retrieval system improving effectiveness
7,1,17496,O01-2005,7,13.749449,retrieval system improving effectiveness
8,1,82490,1998.sigirconf_conference-98.33,8,13.735541,retrieval system improving effectiveness
9,1,124801,2006.ipm_journal-ir0volumeA42A3.2,9,13.569263,retrieval system improving effectiveness


### Step 5: Persist the run file for subsequent evaluations

The output of a prototypical retrieval system is a run file. This run file can later (optimally in a different notebook) be statistically evaluated.

In [21]:
persist_and_normalize_run(run, system_name='bm25-baseline', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
